In [ ]:
#BASIC MODEL- CATEGORY 1
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import logging

celsisus_q=np.array([-40,-10,0,8,15,22,38])
fahrenheit_a=np.array([-40,-14,32,46,59,72,100])

model=keras.models.Sequential()
model.add(keras.layers.Dense(1,input_shape=[1]))

model.compile(loss='mse',optimizer=keras.optimizers.Adam(0.1))
his=model.fit(celsisus_q,fahrenheit_a,epochs=1000)

model.save("./basic_model.h5")

In [ ]:
#IRIS- CATEGORY 2
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import tensorflow.keras as keras
from sklearn.preprocessing import OneHotEncoder

data=load_iris()
X,y=data["data"],data["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
enc = OneHotEncoder()
y_train_label = enc.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test_label = enc.fit_transform(y_test.reshape(-1, 1)).toarray()

model=keras.models.Sequential()
model.add(keras.layers.Dense(128,activation="relu"))
model.add(keras.layers.Dense(64,activation="relu"))
model.add(keras.layers.Dense(32,activation="relu"))
model.add(keras.layers.Dense(16,activation="relu"))
model.add(keras.layers.Dense(8,activation="relu"))
model.add(keras.layers.Dense(3,activation="softmax"))

checkpoint_path = './iris_model.ckpt'
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True,
                                                save_best_only=True, monitor='val_loss', verbose=1)
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["acc"])
model.fit(X_train,y_train_label,epochs=20,callbacks=[checkpoint],validation_data=(X_test,y_test_label))
model.save("./iris_model.h5")
model.load_weights(checkpoint_path)
model.evaluate(X_test,y_test_label)

In [ ]:
#MNIST- CATEGORY 2
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import tensorflow.keras as keras
import tensorflow as tf

mnist=keras.datasets.mnist

(x_train,y_train),(x_test,y_test) =mnist.load_data()
x_train=x_train/255.0
x_test=x_test/255.0

print(x_train.shape)
model=keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(128,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax"))

checkpoint_path = './mnist_model.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True,
                                                save_best_only=True, monitor='val_loss', verbose=1)
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=['accuracy'])
his=model.fit(x_train,y_train,epochs=15,validation_data=(x_test,y_test),callbacks=[checkpoint])
model.save("./mnist_model.h5")
model.evaluate(x_test,y_test)

In [ ]:
#FASHION MNIST- CATEGORY 2
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import tensorflow.keras as keras
import tensorflow as tf
import  numpy as np

mnist=keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train=x_train.astype(np.float32)/255
x_test=x_test.astype(np.float32)/255
x_train=x_train.reshape(-1,28,28,1)
x_test=x_test.reshape(-1,28,28,1)

print(x_train.shape)
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(64,kernel_size=2,padding="same",activation="relu",input_shape=[28,28,1]))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(128,(3,3),activation="relu"))
model.add(keras.layers.MaxPooling2D(2,2))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax"))

checkpoint_path = './fashion_mnist_model.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True,
                                                save_best_only=True, monitor='val_loss', verbose=1)

model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=['accuracy'])
his=model.fit(x_train,y_train,epochs=15,callbacks=[checkpoint])
model.evaluate(x_test,y_test)
model.save("./fashion_mnist.h5")

In [ ]:
#HORSE OR HUMAN-CATEGORY 3
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import urllib
import zipfile
import tensorflow.keras as keras
import tensorflow.keras.applications as applications

#dataset,info=tfds.load(name='horses_or_humans',split=tfds.Split.TRAIN,with_info=True)
#test_dataset=tfds.load(name='horses_or_humans',split=tfds.Split.TEST)

url_train="https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip"
url_test="https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip"

urllib.request.urlretrieve(url_train,'horse-or-human.zip')
local_zip='horse-or-human.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('tmp/house-or-human/')
zip_ref.close()

urllib.request.urlretrieve(url_test,'testdata.zip')
local_zip='testdata.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('tmp/testdata/')
zip_ref.close()

train_dgen=ImageDataGenerator(rescale=1/255)
valid_dgen=ImageDataGenerator(rescale=1/255)

train_generator=train_dgen.flow_from_directory('tmp/house-or-human/'
      ,batch_size=128,class_mode='binary',target_size=(300,300))
valid_generator=valid_dgen.flow_from_directory('tmp/testdata/'
      ,batch_size=128,class_mode='binary',target_size=(300,300))

pre_trained_vgg = applications.VGG16(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
pre_trained_vgg.trainable = False

model = keras.models.Sequential()
model.add(pre_trained_vgg)
model.add(keras.layers.Flatten())
#model.add(keras.layers.Dense(64, activation='relu',kernel_regularizer = keras.regularizers.l1_l2(l1=0.001,l2=0.001))) # 추가 전 :0.97
#model.add(keras.layers.Dropout(0.2)) # 추가 전: 0.97
model.add(keras.layers.Dense(512, activation='relu',kernel_regularizer = keras.regularizers.l1_l2(l1=0.001,l2=0.001)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.summary()

checkpoint_path = './best_performed_model.ckpt'
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True,
                                                save_best_only=True, monitor='val_loss', verbose=1)

model.compile(optimizer=keras.optimizers.Adam(lr=0.02),loss='binary_crossentropy',metrics=['acc'])
model.fit(train_generator,epochs=20,validation_data=valid_generator,callbacks=[checkpoint]) # epoch 늘리기


model.save("./horse_or_human.h5")
model.load_weights("./horse_or_human.h5")
model.evaluate(valid_generator)

In [ ]:
#CAT_DOG-CATEGORY 3
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras as keras
import tensorflow as tf
import  numpy as np
import zipfile
import os

""""!wget --no-check-certificate \
https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
-O /tmp/cats_and_dogs_filtered.zip"""


local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

# 기본 경로
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
train_dgen=ImageDataGenerator(rescale=1/255)
valid_dgen=ImageDataGenerator(rescale=1/255)
train_generator=train_dgen.flow_from_directory(train_dir
      ,batch_size=32,class_mode='binary',target_size=(300,300))
valid_generator=valid_dgen.flow_from_directory(validation_dir
      ,batch_size=32,class_mode='binary',target_size=(300,300))

#x_train=x_train.reshape(-1,28,28,1)
#x_test=x_test.reshape(-1,28,28,1)


pre_trained_vgg = keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
pre_trained_vgg.trainable = False

"""model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  #tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])"""
model = tf.keras.models.Sequential()
model.add(pre_trained_vgg)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512,activation="relu"))
model.add(keras.layers.Dense(1,activation="sigmoid"))
model.summary()

checkpoint_path = './cat_vs_dog_model.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True,
                                                save_best_only=True, monitor='val_loss', verbose=1)

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])
his=model.fit(train_generator,epochs=10,validation_data=valid_generator,callbacks=[checkpoint])
model.save("./cat_dog_model.h5")

In [ ]:
#RPS-CATEGORY 3
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras as keras
import tensorflow as tf
import  numpy as np
import zipfile
import os
import urllib

url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
urllib.request.urlretrieve(url, 'rps.zip')
local_zip = 'rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/')
zip_ref.close()

base_dir = "tmp/rps/"
train_dgen=ImageDataGenerator(rescale=1/255)
valid_dgen=ImageDataGenerator(rescale=1/255)
train_generator=train_dgen.flow_from_directory(base_dir
      ,batch_size=32,class_mode='categorical',subset="training",target_size=(300,300))
valid_generator=valid_dgen.flow_from_directory(base_dir
      ,batch_size=32,class_mode='categorical',subset="validation",target_size=(300,300))

#x_train=x_train.reshape(-1,28,28,1)
#x_test=x_test.reshape(-1,28,28,1)


pre_trained_vgg = keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
pre_trained_vgg.trainable = False

model = tf.keras.models.Sequential()
model.add(pre_trained_vgg)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512,activation="relu"))
model.add(keras.layers.Dense(3,activation="softmax"))
model.summary()

checkpoint_path = './rps_model.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True,
                                                save_best_only=True, monitor='val_loss', verbose=1)

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])
his=model.fit(train_generator,epochs=10,validation_data=valid_generator,callbacks=[checkpoint])
model.save("./rps_model.h5")

In [ ]:
#SACRASM - CATEGORY 4
import json
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

url="https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json"
urllib.request.urlretrieve(url,'sarcasm.json')

with open('sarcasm.json','r') as d:
  datas=json.load(d)

sentences=[]
labels=[]

for data in datas:
  sentences.append(data['headline'])
  labels.append(data['is_sarcastic'])

train_size = 21367
train_x=sentences[:train_size]
train_y=labels[:train_size]
valid_x=sentences[train_size:]
valid_y=labels[train_size:]

vocab_size=1000
oov_tok="<OOV>"
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)
train_seqences=tokenizer.texts_to_sequences(train_x)
valid_seqences=tokenizer.texts_to_sequences(valid_x)
max_length=120
trunc_type='post'
padding_type='post'
train_padded=pad_sequences(train_seqences,truncating=trunc_type,padding=padding_type, maxlen=max_length)
valid_padded=pad_sequences(valid_seqences,truncating=trunc_type,padding=padding_type, maxlen=max_length)
train_labels=np.asarray(train_y)
valid_labels=np.asarray(valid_y)

print(train_padded.shape)

model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, 16, input_length=max_length),
        keras.layers.Bidirectional(keras.layers.LSTM(32)),
        keras.layers.Dense(24, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')])

model.summary()
checkpoint_path = './best_performed_model.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, 
                                                save_best_only=True, monitor='val_loss', verbose=1)
model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['acc'])
model.fit(train_padded,train_labels,batch_size=2,epochs=10,validation_data=(valid_padded,valid_labels),callbacks=[checkpoint])
model.save("./sarcasm.h5")
model.load_weights(checkpoint_path)
model.evaluate(valid_padded,valid_labels)

In [ ]:
#SUNSPOT WITH LAMDA(LSTM) -CATEGORY 5
import csv
import urllib
import tensorflow.keras as keras
import tensorflow as tf
import pandas as pd
import numpy as np

url="https://storage.googleapis.com/laurencemoroney-blog.appspot.com/Sunspots.csv"
urllib.request.urlretrieve(url,'./sunspots.csv')

data=pd.read_csv("./sunspots.csv")

window_size = 100
batch_size = 256
shuffle_buffer_size = 1000

def train_data_pipeline(series, window_size, batch_size, shuffle_buffer_size):
    series = tf.expand_dims(series, axis=-1)
    data = tf.data.Dataset.from_tensor_slices(series)
    data = data.window(window_size+1, shift=1, drop_remainder=True)
    data = data.flat_map(lambda w: w.batch(batch_size))
    data = data.shuffle(shuffle_buffer_size)
    data = data.map(lambda w: (w[:-1], w[1:]))
    return data.batch(batch_size).prefetch(1)

Sunspots = data['Monthly Mean Total Sunspot Number']
series = np.array(Sunspots)
time = np.arange(0, 3235)
train = train_data_pipeline(series, window_size, batch_size, shuffle_buffer_size)

tf.keras.backend.clear_session()

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding='causal', activation='relu', input_shape=[None, 1]),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.Dense(32, activation="relu"),
  tf.keras.layers.Dense(16, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 25.0) #400
])

model.summary()

def test_data_pipeline(series):
    series = tf.expand_dims(series, axis=-1)
    data = tf.data.Dataset.from_tensor_slices(series)
    data = data.window(window_size, shift=1, drop_remainder=True)
    data = data.flat_map(lambda w: w.batch(window_size))
    data = data.batch(batch_size).prefetch(1)
    return data

test = test_data_pipeline(series)
checkpoint_path = './sunspots_no_lambda_model.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True,
                                                save_best_only=True, monitor='mae', verbose=1)
model.compile(loss="mse", optimizer='adam', metrics=["mae"])
history = model.fit(train, epochs=500,validation_data=test,callbacks=[checkpoint])
model.save("./sunspot_model.h5")

In [ ]:
#SUNSPOT WITHOUT LAMDA(DNN) -CATEGORY 5
import csv
import urllib
import tensorflow.keras as keras
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow as tf
print(tf.__version__)

import numpy as np
import matplotlib.pyplot as plt
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

url="https://storage.googleapis.com/laurencemoroney-blog.appspot.com/Sunspots.csv"
urllib.request.urlretrieve(url,'./sunspots.csv')

import csv
time_step = []
sunspots = []

with open('./sunspots.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  next(reader)
  for row in reader:
    sunspots.append(float(row[2]))
    time_step.append(int(row[0]))

series = np.array(sunspots)
time = np.array(time_step)
plt.figure(figsize=(10, 6))
plot_series(time, series)

split_time = 3000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

window_size = 60
batch_size = 32
shuffle_buffer_size = 1000

def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[1]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, input_shape=[window_size], activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1)
])
checkpoint_path = "./sunspots_no_lambda_model.ckpt"
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True,save_best_only=True, monitor='mae', verbose=1)
model.compile(loss="mse",metrics=["mae"], optimizer=tf.keras.optimizers.SGD(lr=1e-7, momentum=0.9))
model.fit(dataset,epochs=500,callbacks=[checkpoint])

forecast=[]
for time in range(len(series) - window_size):
  forecast.append(model.predict(series[time:time + window_size][np.newaxis]))

forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:,-1]


plt.figure(figsize=(10, 6))

plot_series(time_valid, x_valid)
plot_series(time_valid, results)

tf.keras.metrics.mean_absolute_error(x_valid, results).numpy()